KoNLPy 의 Komoran 을 이용하여 라라랜드 영화 리뷰를 토크나이징 한 텍스트를 `lalaland_komoran.txt` 에, 원문을 `lalaland.txt` 에 저장하였습니다. 총 15,595 건의 영화평입니다.

In [1]:
# Komoran tokenized La La Land review
with open('./lalaland_komoran.txt', encoding='utf-8') as f:
    sents = [sent.strip() for sent in f]

with open('./lalaland.txt', encoding='utf-8') as f:
    texts = [sent.strip() for sent in f]

print(len(sents), len(texts))

15595 15595


TextRank 에서는 word cooccurrence graph 를 만들 때 명사와 동사 등을 이용할 것을 제안하였습니다. 이는 조사나 관사와 같이 의미를 지니지 않으면서도 자주 이용되는 단어들이 높은 PageRank 를 가지게 되는 것을 방지하기 위해서입니다.

Komoran 에서의 명사 (NN), 어근 (XR), 형용사 (VA), 동사 (VV) 만을 이용하여 word cooccurrence graph 를 만듭니다. window 를 -1 로 설정하면 한 문장에서 얼마나 떨어져 있던지 상관없이 cooccurrence 를 계산하며, window 가 1 보다 클 경우에는 해당 간격만큼만 떨어진 단어들 간에만 cooccurrence 를 인정합니다.

학습 결과 `영화`, `음악`, `꿈`, `마지막` 같은 라라랜드의 엔딩을 의미하는 단어들이 핵심 단어로 선택되었습니다.

In [2]:
from textrank import KeywordSummarizer

def komoran_tokenize(sent):
    words = sent.split()
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

keyword_extractor = KeywordSummarizer(
    tokenize = komoran_tokenize,
    window = -1,
    verbose = False
)
keywords = keyword_extractor.summarize(sents, topk=30)
for word, rank in keywords:
    print('{} ({:.3})'.format(word, rank))

영화/NNG (1.73e+02)
보/VV (1.29e+02)
좋/VA (65.5)
하/VV (52.0)
것/NNB (47.4)
같/VA (45.4)
영화/NNP (43.8)
음악/NNG (43.6)
꿈/NNG (41.4)
있/VV (40.8)
없/VA (35.9)
마지막/NNG (31.9)
수/NNB (30.1)
사랑/NNG (28.3)
아름답/VA (26.5)
현실/NNG (24.8)
되/VV (23.9)
노래/NNG (23.4)
생각/NNG (23.2)
스토리/NNP (21.4)
번/NNB (20.3)
거/NNB (19.7)
최고/NNG (19.2)
때/NNG (19.1)
사람/NNG (19.0)
여운/NNP (17.5)
뮤지컬/NNP (16.9)
나오/VV (16.5)
듯/NNB (16.1)
영상미/NNG (16.0)


만약 모든 단어를 이용할 경우에는 조사 (JKG, JX, JKS, ... ) 어미 (EC, EP, ... ) 등이 핵심 단어로 선택됩니다. 이는 word cooccurrence graph 는 사실상 출현빈도가 높은 단어들이 높은 Rank 를 가지도록 유도하기 때문입니다.

In [3]:
keyword_extractor = KeywordSummarizer(
    tokenize=lambda x:x.split(),
    verbose = False
)
keywords = keyword_extractor.summarize(sents, topk=30)
for word, rank in keywords:
    print('{} ({:.3})'.format(word, rank))

ㄴ/ETM (1.24e+02)
고/EC (1.03e+02)
영화/NNG (96.8)
는/ETM (94.6)
이/VCP (92.3)
이/JKS (92.0)
하/XSV (85.2)
에/JKB (79.0)
았/EP (76.1)
보/VV (73.5)
었/EP (72.8)
다/EC (68.3)
을/JKO (64.2)
하/XSA (58.8)
의/JKG (58.4)
도/JX (52.7)
ㄹ/ETM (50.2)
가/JKS (47.2)
게/EC (46.7)
는/JX (42.3)
어/EC (37.9)
좋/VA (37.6)
를/JKO (34.3)
아/EC (33.8)
은/ETM (33.7)
들/XSN (32.6)
은/JX (32.0)
하/VV (29.8)
것/NNB (26.7)
과/JC (26.5)


window 의 크기를 바꾼다 하여도 큰 변화는 없습니다. 약간의 순위 변동은 있지만, 큰 맥락이 변하지는 않습니다.

In [4]:
keyword_extractor = KeywordSummarizer(
    tokenize = komoran_tokenize,
    window = 2,
    verbose = False
)
keywords = keyword_extractor.summarize(sents, topk=30)
for word, rank in keywords:
    print('{} ({:.3})'.format(word, rank))

영화/NNG (1.9e+02)
보/VV (1.5e+02)
좋/VA (80.8)
하/VV (51.2)
음악/NNG (50.8)
영화/NNP (50.3)
것/NNB (44.6)
꿈/NNG (42.5)
같/VA (40.7)
있/VV (40.6)
없/VA (35.5)
마지막/NNG (33.7)
아름답/VA (32.1)
사랑/NNG (30.4)
수/NNB (29.5)
현실/NNG (27.9)
노래/NNG (26.1)
최고/NNG (23.8)
스토리/NNP (23.6)
생각/NNG (23.5)
되/VV (23.1)
번/NNB (22.7)
여운/NNP (22.1)
감동/NNG (19.1)
사람/NNG (18.6)
때/NNG (18.0)
거/NNB (18.0)
지루/XR (17.6)
영상미/NNG (16.8)
재밌/VA (16.3)


핵심 문장을 선택하기 위하여 문장 간 유사도를 계산한 다음, `min_sim` 이상의 유사도를 지니는 문장 간에 adjacent sentence graph 를 만듭니다. 그리고 여기에 PageRank 를 적용하여 핵심 문장을 선택합니다.

텍스트를 출력할 때에는 토크나이징된 문장은 가독이 어렵기 때문에 원 텍스트를 이용하여 출력합니다.

In [5]:
from textrank import KeysentenceSummarizer

summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenize,
    min_sim = 0.5,
    verbose = True
)
keysents = summarizer.summarize(sents)
for idx, rank, komoran_sent in keysents:
    print('#{} ({:.3}) : {}'.format(idx, rank, texts[idx]), end='\n\n')

constructing sentence graph was constructed from 15595 sents
trained TextRank. n sentences = 15595
#5 (2.56e+02) : 시사회 보고 왔어요 꿈과 사랑에 관한 이야기인데 뭔가 진한 여운이 남는 영화예요

#14 (1.17e+02) : 시사회 갔다왔어요 제가 라이언고슬링팬이라서 하는말이아니고 너무 재밌어요 꿈과 현실이 잘 보여지는영화 사랑스런 영화 전 개봉하면 또 볼생각입니당

#17 (94.4) : 시사회에서 보고왔는데 여운쩔었다 엠마스톤과 라이언 고슬링의 케미가 도입부의 강렬한음악좋았고 예고편에 나왓던 오디션 노래 감동적이어서 눈물나왔다ㅠ 이영화는 위플래쉬처럼 꼭 영화관에봐야함 색감 노래 배우 환상적인 영화

#27 (77.3) : 방금 시사회로 봤는데 인생영화 하나 또 탄생했네 롱테이크 촬영이 예술 영상이 넘나 아름답고 라이언고슬링의 멋진 피아노 연주 엠마스톤과의 춤과 노래 눈과 귀가 호강한다 재미를 기대하면 약간 실망할수도 있지만 충분히 훌륭한 영화

#6 (65.9) : 황홀하고 따뜻한 꿈이었어요 imax로 또 보려합니다 좋은 영화 시사해주셔서 감사해요

#18 (56.3) : 시사회 갔다왔는데 실망했어요 너무 기대하면 안될 것 같습니다 꿈 같은 영화 마법 같은 영화는 맞는데 꿈과 마법이 깨지는 순간 이 영화는 어디로 가고 있는가 하는 생각이 들었어요 할 말은 많지만 욕먹을까봐 줄임

#1 (55.7) : 사랑과 꿈 그 흐름의 아름다움을 음악과 영상으로 최대한 담아놓았다 배우들 연기는 두말할것없고

#26 (54.2) : 시사회 봤네요 영화를 보고나면 지금 내 옆에 있는 사람이 소중하게 느껴질것

#60 (51.9) : 최고의 뮤지컬 영화영화를 보는 내내 꿈 속에 있는듯한 황홀함

#51 (50.7) : 이번 부산국제영화제서 봤습니다 정말 역대 100 최고의 영화라 말할수있을 정도로 훌륭하고 신나고 즐겁고 재밌었습니다 두번보고 세번보고 평생소장하고 싶은 영화로 추천합니다

아래의 뉴스 기사에 대하여 3 개의 핵심 문장을 추출합니다.

In [1]:
sents = [
    '오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다',
    '서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다',
    '경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다',
    '이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다',
    '성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다',
    '이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다',
    '5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다',
    '용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기',
    '신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다',
    '김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시 33분께 풀숲에 숨은 성씨가 허공에 난사한 10여발의 총알 중 일부를 왼쪽 어깨 뒷부분에 맞고 쓰러졌다',
    '김 경위는 구급차가 도착했을 때 이미 의식이 없었고 심폐소생술을 하며 병원으로 옮겨졌으나 총알이 폐를 훼손해 오후 7시 40분께 사망했다',
    '김 경위는 외근용 조끼를 입고 있었으나 총알을 막기에는 역부족이었다',
    '머리에 부상을 입은 이씨도 함께 병원으로 이송됐으나 생명에는 지장이 없는 것으로 알려졌다',
    '성씨는 오패산 터널 밑쪽 숲에서 오후 6시 45분께 잡혔다',
    '총격현장 수색하는 경찰들 서울 연합뉴스 이효석 기자 19일 오후 서울 강북구 오패산 터널 인근에서 경찰들이 폭행 용의자가 사제총기를 발사해 경찰관이 사망한 사건을 조사 하고 있다',
    '총 때문에 쫓던 경관들과 민간인들이 몸을 숨겼는데 인근 신발가게 직원 이모씨가 다가가 성씨를 덮쳤고 이어 현장에 있던 다른 상인들과 경찰이 가세해 체포했다',
    '성씨는 경찰에 붙잡힌 직후 나 자살하려고 한 거다 맞아 죽어도 괜찮다 고 말한 것으로 전해졌다',
    '성씨 자신도 경찰이 발사한 공포탄 1발 실탄 3발 중 실탄 1발을 배에 맞았으나 방탄조끼를 입은 상태여서 부상하지는 않았다',
    '경찰은 인근을 수색해 성씨가 만든 사제총 16정과 칼 7개를 압수했다 실제 폭발할지는 알 수 없는 요구르트병에 무언가를 채워두고 심지를 꽂은 사제 폭탄도 발견됐다',
    '일부는 숲에서 발견됐고 일부는 성씨가 소지한 가방 안에 있었다'
]

띄어쓰기 기준으로 adjacent sentence graph 를 만듭니다.

In [2]:
from textrank import KeysentenceSummarizer

summarizer = KeysentenceSummarizer(
    tokenize = lambda x:x.split(),
    min_sim = 0.3,
    verbose = False
)
keysents = summarizer.summarize(sents, topk=3)
for _, _, sent in keysents:
    print(sent)

오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다
총격현장 수색하는 경찰들 서울 연합뉴스 이효석 기자 19일 오후 서울 강북구 오패산 터널 인근에서 경찰들이 폭행 용의자가 사제총기를 발사해 경찰관이 사망한 사건을 조사 하고 있다
용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기


KoNLPy 의 Komoran 을 이용하여 토크나이징과 핵심문장을 한 번에 추출하는 예시입니다.

In [3]:
from konlpy.tag import Komoran

komoran = Komoran()
def komoran_tokenizer(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenizer,
    min_sim = 0.3,
    verbose = False
)
keysents = summarizer.summarize(sents, topk=3)
for _, _, sent in keysents:
    print(sent)

오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다
용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기
신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다


사실 위의 결과를 얻기 위해서는 토크나이저도 제대로 구축할 필요가 없습니다. 어자피 많이 등장한 단어라면 해당 단어를 구성하는 부분어절 (subword) 역시 자주 등장하였을 것이며, 이를 이용한 문장 간 유사도를 측정하여도 비슷하기 때문입니다.

아래는 띄어쓰기 기준으로 나뉘어진 어절에서 3음절의 subwords 를 잘라내는 토크나이저 입니다.

In [4]:
def subword_tokenizer(sent, n=3):
    def subword(token, n):
        if len(token) <= n:
            return [token]
        return [token[i:i+n] for i in range(len(token) - n + 1)]
    return [sub for token in sent.split() for sub in subword(token, n)]

subword_tokenizer('이것은 부분단어의 예시입니다 짧은 어절은 그대로 나옵니다')

['이것은',
 '부분단',
 '분단어',
 '단어의',
 '예시입',
 '시입니',
 '입니다',
 '짧은',
 '어절은',
 '그대로',
 '나옵니',
 '옵니다']

이를 이용하여도 핵심 문장은 위와 동일하게 출력됩니다.

In [5]:
summarizer = KeysentenceSummarizer(
    tokenize = subword_tokenizer,
    min_sim = 0.3,
    verbose = False
)
keysents = summarizer.summarize(sents, topk=3)
for _, _, sent in keysents:
    print(sent)

오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다
용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기
총격현장 수색하는 경찰들 서울 연합뉴스 이효석 기자 19일 오후 서울 강북구 오패산 터널 인근에서 경찰들이 폭행 용의자가 사제총기를 발사해 경찰관이 사망한 사건을 조사 하고 있다


summarizer 의 R 에는 각 문장 별 중요도 (PageRank 값) 가 저장되어 있습니다.

In [6]:
summarizer.R

array([1.76438621, 0.74969733, 1.33782296, 0.61722741, 0.7377122 ,
       1.07534516, 0.62928904, 1.71145208, 1.07601036, 1.13590053,
       0.94446938, 0.67686714, 0.7008805 , 1.02103025, 1.61461996,
       0.76911158, 0.78024047, 0.65793743, 1.02927478, 0.97072522])

문장의 위치에 따라 중요도를 다르게 설정할 수도 있습니다. 뉴스 기사는 대부분 첫 문장이 중요합니다. 실제로 위의 예시에서도 첫 문장이 가장 중요한 핵심 문장으로 선택되었습니다. 만약 마지막 문장이 중요하다고 가정한다면 이러한 정보를 bias 에 추가할 수 있습니다. numpy.ndarray 형태로 bias 를 만듭니다. 마지막 문장이 다른 문장보다 10 배 중요하다고 가정하였습니다. 이를 summarize 함수의 bias 에 입력하면 가장 먼저 맨 마지막 문장이 중요한 문장으로 선택됩니다. 다른 문장들 중에서도 맨 마지막 문장과 비슷할수록 상대적인 중요도가 더 커집니다.

In [7]:
import numpy as np

bias = np.ones(len(sents))
bias[-1] = 10

keysents = summarizer.summarize(sents, topk=3, bias=bias)
for _, _, sent in keysents:
    print(sent)

일부는 숲에서 발견됐고 일부는 성씨가 소지한 가방 안에 있었다
경찰은 인근을 수색해 성씨가 만든 사제총 16정과 칼 7개를 압수했다 실제 폭발할지는 알 수 없는 요구르트병에 무언가를 채워두고 심지를 꽂은 사제 폭탄도 발견됐다
오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다


R 을 다시 확인해보면 PageRank 값이 달라졌음을 확인할 수 있습니다. 상대적인 위치 외에도 특정 단어가 포함된 문장에 preference (bias) 를 더 높게 설정할 수도 있습니다.

In [8]:
summarizer.R

array([1.22183954, 0.51902092, 0.92584783, 0.42671701, 0.50982682,
       0.74430683, 0.43498201, 1.18547126, 0.74505343, 0.78632222,
       0.65347844, 0.46802437, 0.48465947, 0.70684359, 1.11845189,
       0.53125081, 0.53956034, 0.45476333, 3.14941282, 4.39416707])